In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType, DataType
from pyspark.sql.functions import *
from os.path import abspath


spark = SparkSession.builder.\
                    appName(name= 'SQL_SOURCE').\
                    master('local[*]').\
                    config("spark.driver.extraClassPath","/opt/mssql-tools18/msjdbc/sqljdbc42.jar").\
                    config("spark.executor.extraClassPath","/opt/mssql-tools18/msjdbc/sqljdbc42.jar").\
                    getOrCreate()
                    
print(spark)

In [ ]:
mssqlprops = {
            'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver',
            'database': 'adventureworks2019',
            'user': 'sparkuser',
            'password': 'sparkadmin',
            'url':'jdbc:sqlserver://172.28.160.1:65230;database=spark'
}

In [ ]:
flightdatapath = abspath('/home/doug/ProjetosEstudo/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet/flight_parquet/*')
retailpath = abspath('/home/doug/ProjetosEstudo/Spark-The-Definitive-Guide/data/retail-data/all/online-retail-dataset.csv')

parquetschema = StructType([
                            StructField('DEST_COUNTRY_NAME', StringType(), True),
                            StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
                            StructField('count', IntegerType(), True)
                            ])

In [ ]:

retailframe = spark.read.csv(path=retailpath, header=True)
flightframe = spark.read.format('parquet').\
                            load(path=flightdatapath).\
                            withColumnRenamed('DEST_COUNTRY_NAME', 'destine').\
                            withColumnRenamed('ORIGIN_COUNTRY_NAME', 'origin').\
                            withColumnRenamed('count', 'flyquantity')
retailframe.show(2)
flightframe.show(2)

In [ ]:
mssqlcon = "jdbc:sqlserver://172.28.160.1:65230;database=spark"

df = spark.\
        read.\
        format('jdbc').\
        option('url', mssqlcon).\
        option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").\
        option('user', 'sparkuser').\
        option('password', 'sparkadmin').\
        option('query', 
                """select 
                    salesorderid as orderid, 
                    salesorderdetailid as qtdperorder,
                    orderqty as qty,
                    pt.productid,
                    pt.name as productname
                from salestable as st
                    inner join
                    productable as pt
                on st.productid = pt.productid
                """).\
    load()
    
df.explain()


In [ ]:
sampletable = spark.\
                    read.\
                    format('jdbc').\
                    option('url', mssqlcon).\
                    option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").\
                    option('user', 'sparkuser').\
                    option('password', 'sparkadmin').\
                    option('query', 
                            """select top(100)
                                SalesOrderID
                                ,SalesOrderDetailID
                                ,CarrierTrackingNumber
                                ,OrderQty
                                ,st.ProductID as pdid,
                                name,
                                productnumber,
                                productsubcategoryid
                            from salestable as st
                                inner join
                                productable as pt
                            on st.productid = pt.productid
                            order by newid() 
                            """).\
                load().show(5)

In [ ]:
salestable = sampletable = spark.\
                    read.\
                    format('jdbc').\
                    option('url', mssqlcon).\
                    option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").\
                    option('user', 'sparkuser').\
                    option('password', 'sparkadmin').\
                    option('query', 
                            """
                                select * from salestable
                                
                            """).\
                load()
# salestable.distinct().explain()
# salestable.filter('productid in (729, 733)').explain()
salestable.filter('productid in (729, 733)').show()

In [ ]:
salestable.filter('productid in (729, 733)').explain()

In [ ]:
partitionframe =((
    spark.\
        read.\
        format('jdbc').\
        option('url', mssqlcon).\
        option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").\
        option('user', 'sparkuser').\
        option('password', 'sparkadmin').\
        option('dbtable', 'salestable').\
        option('numPartitions', 10).\
        option('partitionColumn', 'salesorderid').\
        option('colname', 'salesorderid').\
        option('lowerBound', '100').\
        option('upperBound','1000').\
    load()
))

partitionframe.count()

# partitionframe.write.saveAsTable('partsales', format= 'parquet', mode= 'overwrite')


In [ ]:
selectpartsales = ((
    
                spark.
                sql(""" select * from partsales """)
                
))

selectpartsales.count()

selectpartsales.rdd.getNumPartitions

In [ ]:
#essa forma de leitura não é a mais adequada
#somente uma query por consulta.
testesales = (
            spark.
            read.
            jdbc(url = mssqlcon,
                table = 'person.person',
                table =  'sales.salesterritory',
                properties = mssqlprops)).show(2)
        

In [ ]:

#criar tabela no banco
partitionframe.write.\
            jdbc(mssqlcon, 'salesprod', 'overwrite',
                properties = {'user':'sparkuser', 
                            'password':'sparkadmin',
                            'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'})